## Install Lib API

In [ ]:
! pip install https://dnaink.jfrog.io/artifactory/dna-ink-pypi/model-fkeywords/0.1.0/model_fkeywords-0.1.0-py3-none-any.whl

In [ ]:
! python -m spacy download pt_core_news_sm

## Import libs

In [1]:
import pandas as pd
from api_model.nlsuper import NlExtractorProcess
from api_model.nlvisualization import NlVisualization

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/tmp/ipykernel_34332/1571986624.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


## Variables

##### Description of process
        whats_process = 'complete'
            return: process all pipeline
        whats_process = 'partial'
            return: findkeywords and process bigrams
        whats_process = 'only_keywords'
            return: findkeywords   

In [2]:
filename = 'LIBERACAO_MARGEM_TESTE'
prefix = 'csv'
prefix_sep = '|'
column_text = 'TRANSCRICAO'
whats_process = 'complete'
#whats_process = 'partial'
#whats_process = 'only_keywords'
id_database = 'ID'
type_find = 'aproximado' # aproximado # fixo
activate_stopwords = 'sim' # não

In [3]:
text_finds = {
    'negatividade': ['a desejar', 'abrir uma reclamacao', 'absurd', 'aguento mais', 'boceta', 'bosta', 'brigaram', 'buceta', 'burocra', 'cansad', 'caralho', 'conflito', 'constrang', 'decepcao', 'decepcionad', 'quanta demora', 'que demora', 'tentando resolver', 'descontent', 'descrenca', 'descrente', 'desrespeit', 'fdp', 'filha da puta', 'filha de uma puta', 'filho da puta', 'filho de uma puta', 'frustracao', 'frustrad', 'humilhacao', 'humilhad', 'ignorad', 'ignoram', 'incompet', 'inferno', 'informa nada', 'injuria', 'ironia', 'ironic', 'irreponsaveis', 'irresponsabilidade', 'irresponsavel', 'ma vontade', 'mal a pior', 'mal atendid', 'mal educad', 'mal respondem', 'mal tratad', 'mau atendid', 'mau educad', 'mau respondem', 'mau tratad', 'merda', 'nao podemos arcar', 'nao posso arcar', 'nao sabe', 'ninguem resolve', 'ofenderam', 'ofendid', 'orrivel', 'pelo amor de deus', 'pessimo', 'pessimu', 'porra', 'poxa', 'pqp', 'puta que pariu', 'reclamacao', 'reclamar', 'ridicul', 'ruim', 'sabe nada', 'sem educacao', 'ta dificil', 'trata mal', 'trata mau', 'tratou mal', 'tratou mau', 'triste', 'vergonh', 'vagabund', 'raiva', 'detest', 'nao quero mais', 'odeio', 'safad', 'deus me livre', 'vergonh'],
    'reputacao': ['advogad', 'consumidor.gov', 'entrar com processo', 'facebook', 'instagram', 'judiciais', 'judicial', 'justica', 'meu direito', 'meus direito', 'ouvidoria', 'pequenas causas', 'processar voces', 'procon', 'reclame aqui', 'reclameaqui', 'twitter'],
    'rechamada': ['ainda nao', 'alguma posicao', 'ate agora', 'ate o momento', 'atraso', 'chamado aberto', 'consigo resolver', 'contato novamente', 'continuo sem acesso', 'de novo', 'demora', 'desde ontem', 'desde semana', 'diversas vezes', 'duas vezes', 'em andamento', 'entrei em contato', 'era pra ter', 'faz um mes', 'faz uma semana', 'ja abri', 'ja enviei', 'ja liguei', 'ja mandei', 'liguei pra central', 'mesmo erro', 'muitas vezes', 'nao chegou', 'nao e a primeira vez', 'ocorrencia aberta', 'posicionamento', 'segunda vez', 'sem sucesso', 'tentativa', 'terceira vez', 'todo dia', 'varias vezes', 'chamado aberto'],
    'satisfacao': ['adorei', 'adoro', 'ageis', 'agil', 'agilidade', 'amei', 'atendimento perfeito', 'bem atendid', 'carisma', 'diferenciad', 'educad', 'eficacia', 'eficas', 'eficaz', 'elogiar', 'elogio', 'excelente atendimento', 'feliz', 'gostei', 'impecavel', 'me ajudou', 'merito', 'motivacao', 'motivad', 'muito 10', 'muito bom', 'muito dez', 'nota 10', 'nota dez', 'otimo atendimento', 'parabenizar', 'parabens', 'pela ajuda', 'personalizad', 'preparad', 'prestativ', 'profissional', 'qualidade', 'rapidez', 'resolveu', 'satisfacao', 'satisfatori', 'satisfeit', 'solucionou', 'ate que enfim', 'ate que em fim'],
    'concorrencia': ['ifood', 'caju', 'vee', 'flash', 'bem', 'up', 'VR', 'ticket', 'alelo', 'swile'],
    'contencao': ['desculpa', 'desculpe', 'sinto muito', 'desculpe a demora', 'lamento', 'lamentamos', 'perdoa', 'perdao', 'pelo ocorrido', 'o ocorrido', 'fique tranquil'],
    'ocorrencia': ['chamado', 'ocorrencia', 'registrada com sucesso'],
    'direcionamento': ['central de atedimento', 'ligar na central', 'somente na central', 'sodexo.com', 'ligue na central'],
    'pedefacil': ['clique no', 'clique em', 'menu servicos', 'do menu', 'pede facil', 'clica no', 'clica em', 'diretamente no site', 'visualizar em', 'no menu', 'menu'],
    'ativo': ['analista pode entrar em contato', 'o analista entrara em contato', 'para te auxiliar na proposta', 'solicitei o contato', 'deixe seu contato']

          }

In [4]:
additional_stop_words = ['porque','bom','dia','tres','três','alo','alô']

In [5]:
df = NlExtractorProcess.call_process(filename, prefix, prefix_sep,\
                                     column_text, whats_process,\
                                     text_finds, id_database, type_find,\
                                     additional_stop_words, activate_stopwords)

read file /opt/dna/find-keywords/datalake/csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5002 entries, 0 to 5001
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CPF           5002 non-null   int64  
 1   DURATION      5002 non-null   int64  
 2   ENDTIME       5002 non-null   object 
 3   EXTERNALID    5002 non-null   object 
 4   ID            5002 non-null   int64  
 5   STARTTIME     5002 non-null   object 
 6   MONOCHANNEL   4 non-null      object 
 7   SEQUENCIA     4997 non-null   float64
 8   CANAL         4997 non-null   object 
 9   TRANSCRICAO   4997 non-null   object 
 10  AGENT         5002 non-null   object 
 11  WORKGROUP     5002 non-null   object 
 12  CATEGORIES    5002 non-null   object 
 13  TOPICS        4964 non-null   object 
 14  RECHAMADAS    0 non-null      float64
 15  URA_LOG       5002 non-null   int64  
 16  FILA          4310 non-null   object 
 17  VQ            3280 non-nu

22/06/12 20:55:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


root
 |-- ID: string (nullable = true)
 |-- TRANSCRICAO: string (nullable = true)

None
remove null values of dataset


count rows after remove null values 4315
+-------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ID     |TRANSCRICAO                                                                                                                                                           |
+-------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|6837840|sei                                                                                                                                                                   |
|6838011|atendimento momento momento gentileza                                                                                                                                 |
|6838011|atendimento nome posso ajudar                                    

+-------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+-----------------+-------------------------+
|ID     |TRANSCRICAO                                                                                                                                                           |countent_word|countent_bigram  |countent_trigram         |
+-------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+-----------------+-------------------------+
|6837840|sei                                                                                                                                                                   |sei          |null             |null                     |
|6838011|atendimento momento momento gentileza              

In [6]:
df.head()

,CPF,DURATION,ENDTIME,EXTERNALID,ID,STARTTIME,MONOCHANNEL,SEQUENCIA,CANAL,TRANSCRICAO,...,ocorrencia,direcionamento,pedefacil,ativo,numbers_words,ID,TRANSCRICAO,countent_word,countent_bigram,countent_trigram
0,40361829,39,03MAR2022:11:47:38,DE59V9JKTH2GP6JKP00AEM2EPK029HK4,6837807,03MAR2022:11:46:59,NaN,0.0,LEFT,,...,[],[],[],[],1,6837840,sei,sei,None,None
1,40361829,9,03MAR2022:11:50:10,DE59V9JKTH2GP6JKP00AEM2EPK029HLK,6837840,03MAR2022:11:50:01,NaN,0.0,RIGHT,,...,[],[],[],[],1,6838011,atendimento momento momento gentileza,momento,momento gentileza,momento momento gentileza
2,40361829,583,03MAR2022:12:01:33,DE59V9JKTH2GP6JKP00AEM2EPK029HNV,6838011,03MAR2022:11:51:50,NaN,0.0,LEFT,,...,[],[],[],[],1,6838011,atendimento nome posso ajudar,atendimento,nome posso,nome posso ajudar
3,40361829,9,03MAR2022:11:50:10,DE59V9JKTH2GP6JKP00AEM2EPK029HLK,6837840,03MAR2022:11:50:01,NaN,1.0,LEFT,sei,...,[],[],[],[],1,6838011,nome nome jose desde liguei solicitando cancelar cartao cancelar cartao consignado exclusao marta menina disse havia solicitado estado cancelamento exclusao liberacao,exclusao,cancelar cartao,nome nome jose
4,40361829,583,03MAR2022:12:01:33,DE59V9JKTH2GP6JKP00AEM2EPK029HNV,6838011,03MAR2022:11:51:50,NaN,1.0,RIGHT,,...,[],[],[],[],1,6838011,hum,hum,None,None


In [7]:
filename = 'LIBERACAO_MARGEM'
column_filter = 'negatividade' # deixar esse campo vazio caso queira ver tudo
column_text = 'TRANSCRICAO'
whats_process = 'trigram' # trigram or bigram

In [8]:
NlVisualization.wordCloud_Topics(filename,column_filter,column_text,whats_process)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive//LIBERACAO_MARGEM_tratado.csv'

In [9]:
NlVisualization.plot_10_most_common_words(filename,column_filter,column_text)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive//LIBERACAO_MARGEM_tratado.csv'